In [3]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('order_details.csv', 'order_details'),
    ('orders.csv', 'orders'),
    ('pizza_types.csv', 'pizza_types'),
    ('pizzas.csv', 'pizzas')
]
# Connect to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='**********',
    database='pizzasales'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = r"D:\Datasets\pizza_sales"

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)

    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path,encoding='unicode_escape')

    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)

    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()

# Close the connection
conn.close()

Processing order_details.csv
NaN values before replacement:
order_details_id    0
order_id            0
pizza_id            0
quantity            0
dtype: int64

Processing orders.csv
NaN values before replacement:
order_id    0
date        0
time        0
dtype: int64

Processing pizza_types.csv
NaN values before replacement:
pizza_type_id    0
name             0
category         0
ingredients      0
dtype: int64

Processing pizzas.csv
NaN values before replacement:
pizza_id         0
pizza_type_id    0
size             0
price            0
dtype: int64

